In [6]:
import numpy
import pandas as pd
df=pd.read_excel("PURE DATA SET.xls")
df=df.loc[(df!=0).all(1)]
df_function=df.iloc[:,3:25]
df_target=df.iloc[:,1]
def cal_pop_fitness(equation_inputs, pop):
    fitness = numpy.sum(pop*equation_inputs, axis=1)
    return fitness

def select_mating_pool(pop, fitness, num_parents):
    parents = numpy.empty((num_parents, pop.shape[1]))
    for parent_num in range(num_parents):
        max_fitness_idx = numpy.where(fitness == numpy.max(fitness))
        max_fitness_idx = max_fitness_idx[0][0]
        parents[parent_num, :] = pop[max_fitness_idx, :]
        fitness[max_fitness_idx] = -99999999999
    return parents

def crossover(parents, offspring_size):
    offspring = numpy.empty(offspring_size) 
    crossover_point = numpy.uint8(offspring_size[1]/2)
    for k in range(offspring_size[0]):
        parent1_idx = k%parents.shape[0]
        parent2_idx = (k+1)%parents.shape[0]
        offspring[k, 0:crossover_point] = parents[parent1_idx, 0:crossover_point]
        offspring[k, crossover_point:] = parents[parent2_idx, crossover_point:]
    return offspring

def mutation(offspring_crossover, num_mutations=1):
    mutations_counter = numpy.uint8(offspring_crossover.shape[1] / num_mutations)
    for idx in range(offspring_crossover.shape[0]):
        gene_idx = mutations_counter - 1
        for mutation_num in range(num_mutations):
            random_value = numpy.random.uniform(-1.0, 1.0, 1)
            offspring_crossover[idx, gene_idx] = offspring_crossover[idx, gene_idx] + random_value
            gene_idx = gene_idx + mutations_counter
    return offspring_crossover
equation_matrix=df_function.values
result=[]
for equation_inputs in equation_matrix:
    equation_inputs=numpy.log(equation_inputs)     #!!!!!!!!!!!!!!!!!!!!
    #print(equation_inputs)
    num_weights = len(equation_inputs)
    sol_per_pop = 8
    num_parents_mating = 4
    pop_size = (sol_per_pop,num_weights)
    new_population = numpy.random.uniform(low=-4.0, high=4.0, size=pop_size)
    best_outputs = []
    num_generations = 1000
    for generation in range(num_generations):
        fitness = cal_pop_fitness(equation_inputs, new_population)
        best_outputs.append(numpy.max(numpy.sum(new_population*equation_inputs, axis=1)))
        parents = select_mating_pool(new_population, fitness, 
                                        num_parents_mating)
        offspring_crossover = crossover(parents,
                                        offspring_size=(pop_size[0]-parents.shape[0], num_weights))
        offspring_mutation = mutation(offspring_crossover, num_mutations=2)
        new_population[0:parents.shape[0], :] = parents
        new_population[parents.shape[0]:, :] = offspring_mutation
    fitness = cal_pop_fitness(equation_inputs, new_population)
    best_match_idx = numpy.where(fitness == numpy.max(fitness))
    #print("Best solution : ", numpy.exp(new_population[best_match_idx[0][0], :]))
    #print("Best solution fitness : ", fitness[best_match_idx])
    result.append(new_population[best_match_idx,:][0][0])
result=numpy.array(result)
result1=numpy.exp(result)
#result1=numpy.log(result)
pd.DataFrame(result1).to_excel("output.xlsx")

In [7]:
data = pd.read_excel (r'output.xlsx')
print (data)

     Unnamed: 0          0          1  ...         19         20            21
0             0  15.355212   2.181154  ...  25.782298   0.234098  5.224271e+01
1             1  41.075767   6.355812  ...  31.857805  45.607388  5.034928e-01
2             2  15.865893  37.514547  ...  22.828950   0.362938  4.926207e-01
3             3   9.925434  51.406208  ...   0.873639   0.092353  2.911599e-01
4             4   0.253752   4.804137  ...  19.080274  20.077735  4.876396e-02
..          ...        ...        ...  ...        ...        ...           ...
111         111   9.727263   2.824108  ...   2.454291   0.935212  9.568643e+00
112         112   1.178482  35.149255  ...   0.075083   0.037941  4.954149e-01
113         113   1.640927   0.594408  ...  54.009211   0.102091  5.990707e-02
114         114  13.924884   0.104966  ...   1.219481   3.608855  5.442787e-04
115         115   7.848249   0.047682  ...  49.004320   1.702532  1.334372e-08

[116 rows x 23 columns]


In [8]:


a=[10,20,30]
b=numpy.log(a)
print(b)
c=numpy.exp(b)
print(c)


[2.30258509 2.99573227 3.40119738]
[10. 20. 30.]


In [ ]:
pip install pynverse


  Created wheel for pynverse: filename=pynverse-0.1.4.4-cp36-none-any.whl size=6350 sha256=5427a6f8bf467e133270cf714ec26bf45699d0e76368780f4ff9a20f0bb5e911
  Stored in directory: /root/.cache/pip/wheels/80/ab/86/ec93f37613f012ba3e65c2bbd67e5c54cbc00d237574100c24
Successfully built pynverse


In [ ]:
from pynverse import inversefunc
a=[10,20,30]
b=numpy.log(a)
print(b)
c=numpy.log(numpy.exp(b))
print(c)
res = numpy.exp(b.cumsum())

print(res)


[2.30258509 2.99573227 3.40119738]
[2.30258509 2.99573227 3.40119738]
[  10.  200. 6000.]
